<a href="https://colab.research.google.com/github/panghanwu/tibame_project/blob/main/recommendation_oop_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 模糊搜尋功能 v2
---

### 會使用到的套件
- py2neo
- pandas
- numpy

In [1]:
# py2neo是python控制neo4j的套件
# Colab並未內建需要另外安裝
!pip install py2neo

     |████████████████████████████████| 194kB 10.2MB/s 
     |████████████████████████████████| 2.6MB 18.4MB/s 
     |████████████████████████████████| 348kB 50.2MB/s 
     |████████████████████████████████| 153kB 47.9MB/s 
     |████████████████████████████████| 204kB 47.9MB/s 
  Created wheel for neotime: filename=neotime-1.7.4-cp36-none-any.whl size=20543 sha256=39763a70d1478dccb42715d2cca8b54be1833bc1afa8c3452c0df30b2d8a13a5
  Stored in directory: /root/.cache/pip/wheels/d4/b4/c4/ab867cd0ac9e667f8cd269f9364fd1cb8ee75fe50f5ce97c79
Successfully built neotime
ERROR: jupyter-console 5.2.0 has requirement prompt-toolkit<2.0.0,>=1.0.0, but you'll have prompt-toolkit 2.0.10 which is incompatible.
ERROR: ipython 5.5.0 has requirement prompt-toolkit<2.0.0,>=1.0.4, but you'll have prompt-toolkit 2.0.10 which is incompatible.
  Found existing installation: prompt-toolkit 1.0.18
    Uninstalling prompt-toolkit-1.0.18:
      Successfully uninstalled prompt-toolkit-1.0.18


In [2]:
import numpy as np
import pandas as pd
import py2neo as neo


class Neo4jRecomBot():
    # 1. __init__: 連上Neo4j伺服器
    # 2. fussy_search: 模糊搜尋
    # 3. same_search: 同款搜尋
    # 4. fit_search: 穿搭搜尋
    def __init__(self, sever_link, password, word2vec_path):
        # 載入圖資料庫
        self.graph = neo.Graph(sever_link, password=password)
        # 提取出產品清單
        self.product_list = list(neo.NodeMatch(self.graph, labels=frozenset(['Product'])))
        
        # 載入語料詞向量檔
        cloth_model_df = pd.read_csv(word2vec_path)
        cloth_vec = {}
        for i in range(len(cloth_model_df)):
            vector = cloth_model_df['vec'][i]
            vector = vector.replace('\n','')
            vector = vector.replace('[','')
            vector = vector.replace(']','')
            vector = np.fromstring(vector, sep=' ')
            cloth_vec[cloth_model_df['cht'][i]] = vector

        self.cloth_vec = cloth_vec
    

    def fussy_search(self, keyword, gender=None):
        assert gender in ['man', 'woman', None]
        
        # 依據性別更改商品節點清單
        if gender == 'man':
            search_list = [x for x in self.product_list if x['sn'][0]=='M']
        elif gender == 'woman':
            search_list = [x for x in self.product_list if x['sn'][0]=='F']
        else:
            search_list = self.product_list

        # 把辨識描述用word2vec轉成300為的詞向量
        key_vec = np.zeros(300)
        for d in keyword:
            # 所有詞向量加總
            key_vec += self.cloth_vec[d]

        # 把產品詞向量存成矩陣
        pro_vec = np.empty((len(search_list),300))
        for i, n in enumerate(search_list):
            str_vec = n['vector']
            pro_vec[i] = np.fromstring(str_vec, sep=' ')
        
        # 找出夾角最小（最大cos）商品的索引
        dot  = np.dot(key_vec, pro_vec.T)
        norm = np.linalg.norm(key_vec) * np.linalg.norm(pro_vec, axis=1)
        cos  = dot / norm
        idx  = np.argmax(cos)
        sco  = np.max(cos)

        return (search_list[idx],
                search_list[idx]['sn'], 
                search_list[idx]['name'], 
                search_list[idx]['image_url'],
                round(sco, 3))
        
    # 輸入商品節點回傳同款商品
    def same_search(self, product):
        same_relate = list(neo.RelationshipMatch(self.graph, 
                                                 nodes=[product], 
                                                 r_type='SAME'))
        print(same_relate)
        same_node = same_relate[0].end_node
        if same_node != []:
            return (same_node,
                    same_node['sn'],
                    same_node['name'],
                    same_node['image_url'])
        else:
            return '無同款'   

    def fit_search(self, product, top=1):
        fit_relate = neo.RelationshipMatch(self.graph, nodes=[product], r_type='FIT')
        fit_rank = list(fit_relate.order_by('_.score DESC'))
        fit_score = fit_rank[top-1]['score']
        fit_node = fit_rank[top-1].end_node
        return (fit_node, 
                fit_score,
                fit_node['sn'],
                fit_node['name'],
                fit_node['image_url'])


### 範例

In [3]:
# 建立物件
sever_link = 'bolt://52.91.118.4:33083'
pws = 'sevenths-harpoon-jails'
word2vec_path = 'cloth_word_vec.csv'

recommdation = Neo4jRecomBot('bolt://52.91.118.4:33083', 'sevenths-harpoon-jails', word2vec_path)

recommdation.graph

Graph('bolt://neo4j@52.91.118.4:33083')

### 模糊搜尋

In [4]:
# 隨機生成特徵描述

keyword = []
word_list = list(recommdation.cloth_vec)
word_num = len(word_list)

num = 3
ridx = np.random.randint(0, word_num, num)


for i in ridx:
    word = word_list[i]
    keyword.append(word)

keyword

['夾克', '連身衣', '女裝襯衫']

In [5]:
gender = 'woman'

main_recom, main_sn, main_name, main_url, match_score = recommdation.fussy_search(keyword, gender=gender)

main_sn, main_name, match_score

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:52: DeprecationWarning: string or file could not be read to its end due to unmatched data; this will raise a ValueError in the future.


('FL06', '系列-高腰牛仔緊身褲', 0.055)

### 穿搭推薦

In [6]:
# 一階穿搭
fit_recom_first, fit_score_f, fit_sn_f, fit_name_f, fit_url_f = recommdation.fit_search(main_recom, top=1)

print('一階穿搭:', fit_name_f, '分數:', fit_score_f)

一階穿搭: Soft RichV領針織長上衣 分數: 0.849


In [7]:
# 二階穿搭
fit_recom_second, fit_score_s, fit_sn_s, fit_name_s, fit_url_s = recommdation.fit_search(fit_recom_first, top=2)

print('二階穿搭:', fit_name_s, '分數:', fit_score_s)

二階穿搭: 系列-高腰牛仔緊身褲 分數: 0.849


### 同款搜尋

In [8]:
same_recom, same_sn, same_name, same_url = recommdation.same_search(main_recom)

same_sn, same_name

[SAME(Node('Product', description='必備1件的基本款牛仔緊身褲。高腰版型，可拉長腿部視覺效果。深藍', image_url='https://im.uniqlo.com/images/tw/gu/pc/goods/327815/item/68_327815.jpg', name='系列-高腰牛仔緊身褲', sn='FL06', vector='0.8623438761569560.31698289490304897.1103572025895121.1112676961347460.166635485365986821.9409478027373552-1.0903679877519608-1.17968638613820082.0183245921507478-0.24691436812281609-0.230401667300611730.015431974083185196-0.0258632879704237-0.28835577890276911.0763966180384160.5483705615624785-1.0812293803028297-0.08730277733411640.3919586273841560.62363846180960540.16109015521942638-0.064816567348316311.12762435339391230.2552086854702793-0.198929798789322380.96876326389610770.158192310482263570.2773163001984358-0.50248623639345170.08042053168173880.5929635651409626-0.09220483759418130.63206293061375620.9249431106727570.403417314169928430.83259260025806730.9992027275729924-1.7249435791745782-0.57714936090633270.8897751993063139-0.09815982799045742-0.34493593405932190.1591554624028504-0.014143472944

('FL05', '系列-高腰牛仔緊身褲')